In [33]:
import pandas as pd
from pulp import * 
import matplotlib.pyplot as plt
from itertools import chain, repeat

### Declaration and Defenition

The input values provided signify the demand in terms of **number of boxes**. The first step here is to convert the number of boxes to **number of workers required**. After this, **a circular list of days** is created to find the **number of working days per shift** and the **number of workers off shift each day**. 

In [34]:
# Demand per day
demand_boxes=[910, 840, 875, 805, 875, 819, 840]

#Productivity and working hours
pd_weekdays=5
pd_weekends=3
hour_worked=7

# Staff needs per Day
demand_workers=[1, 1, 1, 1, 1, 1, 1]
for i in range (7):
    if i<5:
        demand_workers[i]=demand_boxes[i]/(pd_weekdays*hour_worked)
    else:
        demand_workers[i]=demand_boxes[i]/(pd_weekends*hour_worked)
        
    
# Days of the week
week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Create circular list of days
n_days = [i for i in range(7)]
n_days_c = n_days * 3

# Working days
days_in = [[n_days_c[j] for j in range(i , i + 5)] for i in n_days_c]

# Workers off by shift for each day
shifts_excl = [[n_days_c[j] for j in range(i + 1, i + 3)] for i in n_days_c]

## The problem

A **minimization** problem is defined to minimize the **total number of workers required**. A staff **supply that is equal to or greater than the demand** and a **non negativity consideration** for number of staffs per shift are considered as constraints.

In [35]:
# Initialize Model
model = LpProblem("Minimize Staffing", LpMinimize)
print("Model initialized: {}".format(model))

# Create Variables
week_list = ['Shift:' + i for i in week]
x = LpVariable.dicts('x', n_days, lowBound=0, cat='Integer')

# Define Objective
model += lpSum([x[i] for i in n_days])
print("Model after objective: {}".format(model))

# Add constraints
for d, l_excl, staff in zip(n_days, shifts_excl, demand_workers):
    print("d:{}, l_excel:{}, staff:{}".format(d, l_excl, staff))
    model += lpSum([x[i] for i in n_days if i not in l_excl]) >= staff
print("\nModel after adding constraints: {}".format(model))

Model initialized: Minimize_Staffing:
MINIMIZE
None
VARIABLES

Model after objective: Minimize_Staffing:
MINIMIZE
1*x_0 + 1*x_1 + 1*x_2 + 1*x_3 + 1*x_4 + 1*x_5 + 1*x_6 + 0
VARIABLES
0 <= x_0 Integer
0 <= x_1 Integer
0 <= x_2 Integer
0 <= x_3 Integer
0 <= x_4 Integer
0 <= x_5 Integer
0 <= x_6 Integer

d:0, l_excel:[1, 2], staff:26.0
d:1, l_excel:[2, 3], staff:24.0
d:2, l_excel:[3, 4], staff:25.0
d:3, l_excel:[4, 5], staff:23.0
d:4, l_excel:[5, 6], staff:25.0
d:5, l_excel:[6, 0], staff:39.0
d:6, l_excel:[0, 1], staff:40.0

Model after adding constraints: Minimize_Staffing:
MINIMIZE
1*x_0 + 1*x_1 + 1*x_2 + 1*x_3 + 1*x_4 + 1*x_5 + 1*x_6 + 0
SUBJECT TO
_C1: x_0 + x_3 + x_4 + x_5 + x_6 >= 26

_C2: x_0 + x_1 + x_4 + x_5 + x_6 >= 24

_C3: x_0 + x_1 + x_2 + x_5 + x_6 >= 25

_C4: x_0 + x_1 + x_2 + x_3 + x_6 >= 23

_C5: x_0 + x_1 + x_2 + x_3 + x_4 >= 25

_C6: x_1 + x_2 + x_3 + x_4 + x_5 >= 39

_C7: x_2 + x_3 + x_4 + x_5 + x_6 >= 40

VARIABLES
0 <= x_0 Integer
0 <= x_1 Integer
0 <= x_2 Integer
0 <

## Solution

In [36]:
# Solve Model
model.solve()
# print("Model after solving: {}".format(model))
# print("Model variables: {}".format(model.variables()))
# The status of the solution is printed to the screen
print("Status:", LpStatus[model.status])

# How many workers per day ?
dct_work = {}
for v in model.variables():
    dct_work[int(v.name[-1])] = int(v.varValue)
    print("Shift {} needs {} workers.".format(int(v.name[-1])+1, int(v.varValue)))
    
# Show Detailed Sizing per Day
dict_sch = {}
for day in dct_work.keys():
    dict_sch[day] = [dct_work[day] if i in list_in[day] else 0 for i in n_days]

# Calculate Staff Availability per Day
totals = [] 
for i in n_days:
    total = 0
    for j in n_days:
        total += dict_sch[j][i]
    totals.append(total)

dict_sch[7] = totals
dict_sch[8] = demand_workers
dict_sch[9] = [totals[i] - demand_workers[i] for i in n_days]

df_sch = pd.DataFrame(dict_sch).T
df_sch.columns = week
df_sch.index = week_list + ['Total Workers'] + ['Demand'] + ['Excess Workers']

display(df_sch)

# The optimized objective function value is printed to the screen
print("Total number of Staff = ", pulp.value(model.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/e04c2c3d9c6047d9bf998cfcc7fd873f-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/e04c2c3d9c6047d9bf998cfcc7fd873f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 12 COLUMNS
At line 69 RHS
At line 77 BOUNDS
At line 85 ENDATA
Problem MODEL has 7 rows, 7 columns and 35 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 42 - 0.00 seconds
Cgl0003I 0 fixed, 7 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 7 rows, 7 columns (7 integer (0 of which binary)) and 35 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of 42 found by greedy cover after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 42, 

,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
Shift:Monday,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Shift:Tuesday,0.0,2.0,2.0,2.0,2.0,2.0,0.0
Shift:Wednesday,0.0,0.0,6.0,6.0,6.0,6.0,6.0
Shift:Thursday,12.0,0.0,0.0,12.0,12.0,12.0,12.0
Shift:Friday,5.0,5.0,0.0,0.0,5.0,5.0,5.0
Shift:Saturday,14.0,14.0,14.0,0.0,0.0,14.0,14.0
Shift:Sunday,3.0,3.0,3.0,3.0,0.0,0.0,3.0
Total Workers,34.0,24.0,25.0,23.0,25.0,39.0,40.0
Demand,26.0,24.0,25.0,23.0,25.0,39.0,40.0
Excess Workers,8.0,0.0,0.0,0.0,0.0,0.0,0.0


Total number of Staff =  42.0
